In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Importing packages needed

In [ ]:
import pandas as pd 
import numpy as np

### Importing the data

In [727]:
path = "/content/drive/MyDrive/Omdela/port_harcourt__nigeria__4.80678_7.002953_.csv"
NewTSA = pd.read_csv(path, parse_dates=['created_at'])
NewTSA.rename(columns={'PM2.5_ATM_ug/m3':'pm', 'entry_id':'entryId', 'created_at':'createdAt', 'Temperature_F':'Temperature', 'Humidity_%':'Humidity'}, inplace=True)
NewTSA.set_index('createdAt', inplace=True)

In [728]:
NewTSA.head()

,entryId,Temperature,Humidity,pm
createdAt,,,,
2021-07-26 16:45:59+00:00,198,89,50,40.78
2021-07-26 16:48:03+00:00,199,89,50,40.09
2021-07-26 16:49:59+00:00,200,89,50,37.93
2021-07-26 16:54:03+00:00,201,89,50,33.18
2021-07-26 16:55:59+00:00,202,89,50,37.84


In [730]:
NewTSA.describe()

,entryId,Temperature,Humidity,pm
count,56052.000000,56052.000000,56052.000000,56052.000000
mean,28223.500000,84.821469,65.117766,54.391926
std,16180.962981,4.319505,8.550730,42.205997
min,198.000000,70.000000,29.000000,0.780000
25%,14210.750000,82.000000,61.000000,30.170000
50%,28223.500000,84.000000,68.000000,41.450000
75%,42236.250000,87.000000,71.000000,61.190000
max,56249.000000,108.000000,89.000000,568.080000


In [731]:
NewTSA

,entryId,Temperature,Humidity,pm
createdAt,,,,
2021-07-26 16:45:59+00:00,198,89,50,40.78
2021-07-26 16:48:03+00:00,199,89,50,40.09
2021-07-26 16:49:59+00:00,200,89,50,37.93
2021-07-26 16:54:03+00:00,201,89,50,33.18
2021-07-26 16:55:59+00:00,202,89,50,37.84
...,...,...,...,...
2021-10-25 09:28:43+00:00,56245,87,61,29.17
2021-10-25 09:30:43+00:00,56246,87,61,29.57
2021-10-25 09:32:44+00:00,56247,87,61,28.75


### setting frequency

In [ ]:
#NewTSA = NewTSA.asfreq('b')

In [ ]:
#NewTSA.head()

### Taking care of missing values

In [ ]:
#NewTSA.isna()

In [ ]:
#NewTSA.isna().sum()

In [ ]:
#NewTSA.entry_id = NewTSA.entry_id.fillna(method = 'ffill')
#NewTSA.isna().sum()

In [ ]:
#NewTSA.Temperature_F = NewTSA.Temperature_F.fillna(method = 'ffill')
#NewTSA.isna().sum()

In [ ]:
#NewTSA.Humidity = NewTSA.Humidity.fillna(value = NewTSA.Humidity.mean())
#NewTSA.isna().sum()

In [ ]:
#NewTSA.PM = NewTSA.PM.fillna(method = 'bfill')
#NewTSA.isna().sum()

### Simplifying the dataset

In [732]:
NewTSA['PM2'] = NewTSA.pm

In [733]:
NewTSA.describe()

,entryId,Temperature,Humidity,pm,PM2
count,56052.000000,56052.000000,56052.000000,56052.000000,56052.000000
mean,28223.500000,84.821469,65.117766,54.391926,54.391926
std,16180.962981,4.319505,8.550730,42.205997,42.205997
min,198.000000,70.000000,29.000000,0.780000,0.780000
25%,14210.750000,82.000000,61.000000,30.170000,30.170000
50%,28223.500000,84.000000,68.000000,41.450000,41.450000
75%,42236.250000,87.000000,71.000000,61.190000,61.190000
max,56249.000000,108.000000,89.000000,568.080000,568.080000


In [706]:
#del NewTSA['entry_id'], NewTSA['Temperature'], NewTSA['Humidity']

In [721]:
#NewTSA.describe()

Creating Returns

In [724]:
NewTSA['ret_entryId'] = NewTSA.entryId


### Splitting the data

In [709]:
size = int(len(NewTSA)*0.8)

In [ ]:
df = NewTSA.iloc[:size]

In [ ]:
df_test = NewTSA.iloc[size:]

In [ ]:
df.tail()

In [ ]:
df_test.head()

### White Noise

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa import api as sts
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
sns.set()

In [ ]:
wn = np.random.normal(loc = df.PM2.mean(), scale = df.PM2.std(), size = len(df))

In [ ]:
df['wn'] = wn

In [ ]:
df.describe()

In [ ]:
df.wn.plot(figsize = (20,5))
plt.title('White Noise', size = 24)
plt.show()

In [ ]:
df.PM2.plot(figsize = (20,5))
plt.title('PM', size = 24)
plt.ylim(-100, 200)
plt.show()

### Stationary

In [ ]:
sts.adfuller(df.PM2)

In [ ]:
sts.adfuller(df.wn)

### Seasonality

In [ ]:
s_dec_additive = seasonal_decompose(df.PM2, model = 'additive', freq=50)
s_dec_additive.plot()
plt.show()

In [ ]:
s_dec_multiplicative = seasonal_decompose(df.PM2, model = 'multiplicative', freq=50)
s_dec_multiplicative.plot()
plt.show()

### ACF

In [ ]:
sgt.plot_acf(df.PM2, lags = 40, zero = False)
plt.title('ACF PM2', size = 24)
plt.show()

In [ ]:
sgt.plot_acf(df.wn, lags = 40, zero = False)
plt.title('ACF WN', size = 24)
plt.show()

### PACF

In [ ]:
sgt.plot_pacf(df.PM2, lags = 40, zero = False, method=('ols'))
plt.title('PACF PM2', size = 24)
plt.show()

In [ ]:
sgt.plot_pacf(df.wn, lags = 40, zero = False, method=('ols'))
plt.title('PACF WN', size = 24)
plt.show()

### AR(1) Model

In [ ]:
import statsmodels
statsmodels.__version__
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.arima_model import ARMA
from scipy.stats.distributions import chi2
model_ar = ARMA(df.PM2, order = (1,0))

In [ ]:
result_ar = model_ar.fit()

In [ ]:
result_ar.summary()

### Higher-Lag AR Models

In [ ]:
model_ar_2 = ARMA(df.PM2, order = (2,0))
result_ar_2 = model_ar_2.fit()

In [ ]:
result_ar_2.summary()

In [ ]:
model_ar_3 = ARMA(df.PM2, order = (3,0))
result_ar_3 = model_ar_3.fit()
result_ar_3.summary()

In [ ]:
model_ar_4 = ARMA(df.PM2, order = (4,0))
result_ar_4 = model_ar_4.fit()
result_ar_4.summary()

### LLR Test

In [ ]:
def LLR_test(mod_1, mod_2, DF=1):
    L1 = mod_1.llf
    L2 = mod_2.llf
    LR = (2*(L2-L1))
    p = chi2.sf(LR, DF).round(3)
    return p

### Comparing Higher-Lag AR Models

In [ ]:
LLR_test(result_ar_2, result_ar_3)

In [ ]:
LLR_test(result_ar_3, result_ar_4)

In [ ]:
model_ar_4 = ARMA(df.PM2, order = [4,0])
result_ar_4 = model_ar_4.fit()
print(result_ar_4.summary())
print('LLR_test: ' + str(LLR_test(result_ar_3, result_ar_4)))

In [ ]:
model_ar_5 = ARMA(df.PM2, order = [5,0])
result_ar_5 = model_ar_5.fit()
print(result_ar_5.summary())
print('\nLLR test p-value = ' + str(LLR_test(result_ar_4, result_ar_5)))

In [ ]:
model_ar_6 = ARMA(df.PM2, order = [6,0])
result_ar_6 = model_ar_6.fit()
print(result_ar_6.summary())
print('\nLLR test p-value = ' + str(LLR_test(result_ar_5, result_ar_6)))

In [ ]:
model_ar_7 = ARMA(df.PM2, order = [7,0])
result_ar_7 = model_ar_7.fit()
print(result_ar_7.summary())
print('\nLLR test p-value = ' + str(LLR_test(result_ar_6, result_ar_7)))

In [ ]:
print('LLR_test: ' + str(LLR_test(result_ar, result_ar_7, DF=6)))

### The DF-Test

In [ ]:
sts.adfuller(df.PM2)

### Using Returns

In [ ]:
df['returns'] = df.PM2.pct_change(1).mul(100)
df = df.iloc[1:]

In [ ]:
sts.adfuller(df.returns)

### ACF and PACF Returns

In [ ]:
sgt.plot_acf(df.returns, lags=40, zero=False)
plt.title('ACF Humidity Returns', size=24)
plt.show()

In [ ]:
sgt.plot_pacf(df.returns, lags=40, zero=False, method=('ols'))
plt.title('PACF Humidity Returns', size=24)
plt.show()

### AR(1) for Returns 

In [ ]:
model_ret_ar_1 = ARMA(df.returns, order =(1,0))
results_ret_ar_1 = model_ret_ar_1.fit()
results_ret_ar_1.summary()

### Higher-Lag AR Models for Returns

In [ ]:
model_ret_ar_2 = ARMA(df.returns, order =(2,0))
results_ret_ar_2 = model_ret_ar_2.fit()
results_ret_ar_2.summary()

In [ ]:
LLR_test(results_ret_ar_1, results_ret_ar_2)

In [ ]:
model_ret_ar_3 = ARMA(df.returns, order =(3,0))
results_ret_ar_3 = model_ret_ar_3.fit()
results_ret_ar_3.summary()

In [ ]:
LLR_test(results_ret_ar_2, results_ret_ar_3)

In [ ]:
model_ret_ar_3 = ARMA(df.returns, order =(3,0))
results_ret_ar_3 = model_ret_ar_3.fit()
results_ret_ar_3.summary()

In [ ]:
model_ret_ar_4 = ARMA(df.returns, order =(4,0))
results_ret_ar_4 = model_ret_ar_4.fit()
print(results_ret_ar_4.summary())
print('LLR test: ' + str(LLR_test(results_ret_ar_3, results_ret_ar_4)))

In [ ]:
model_ret_ar_5 = ARMA(df.returns, order =(5,0))
results_ret_ar_5 = model_ret_ar_5.fit()
print(results_ret_ar_5.summary())
print('LLR test: ' + str(LLR_test(results_ret_ar_4, results_ret_ar_5)))

In [ ]:
model_ret_ar_6 = ARMA(df.returns, order =(6,0))
results_ret_ar_6 = model_ret_ar_6.fit()
print(results_ret_ar_6.summary())
print('LLR test: ' + str(LLR_test(results_ret_ar_5, results_ret_ar_6)))

In [ ]:
model_ret_ar_7 = ARMA(df.returns, order =(7,0))
results_ret_ar_7 = model_ret_ar_7.fit()
print(results_ret_ar_7.summary())
print('LLR test: ' + str(LLR_test(results_ret_ar_6, results_ret_ar_7)))

### Normalizing Values

In [ ]:
benchmark = df.PM2.iloc[0]
df['norm'] = df.PM2.div(benchmark).mul(100)
sts.adfuller(df.norm)

In [ ]:
bech_ret = df.returns.iloc[0]
df['norm_ret'] = df.returns.div(bech_ret).mul(100)
sts.adfuller(df.norm_ret)

### Normalizing Returns

In [ ]:
model_norm_ret_ar_1 = ARMA(df.norm_ret, order=(1,0))
results_norm_ret_ar_1 = model_norm_ret_ar_1.fit()
results_norm_ret_ar_1.summary()

In [ ]:
model_norm_ret_ar_7 = ARMA(df.norm_ret, order=(7,0))
results_norm_ret_ar_7 = model_norm_ret_ar_7.fit()
results_norm_ret_ar_7.summary()

### Analysing the Residuals

In [ ]:
df['res_changes'] = result_ar_7.resid

In [ ]:
df.res_changes.mean()

In [ ]:
df.res_changes.var()

In [ ]:
sts.adfuller(df.res_changes)

In [ ]:
sgt.plot_acf(df.res_changes, zero=False, lags=40)
plt.title('ACF of Residuals for Changes', size=24)
plt.show()

In [ ]:
df.res_changes[1:].plot(figsize=(20,5))
plt.title('Residuals of Changes', size=24)
plt.show()

### ACF for Returns

In [ ]:
sgt.plot_acf(df.returns[1:], zero = False, lags = 40)
plt.title('ACF for Returns', size = 24)
plt.show()

### MA(1) for Returns

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
model_ret_ma_1 = ARMA(df.returns[1:], order=(0,1))
results_ret_ma_1 = model_ret_ma_1.fit()
results_ret_ma_1.summary()

### Higher-Lag MA Models for Returns

In [ ]:
model_ret_ma_2 = ARMA(df.returns[1:], order=(0,2))
results_ret_ma_2 = model_ret_ma_2.fit()
print(results_ret_ma_2.summary())
print('\nLLR test p-value = ' + str(LLR_test(results_ret_ma_1, results_ret_ma_2)))

In [ ]:
model_ret_ma_3 = ARMA(df.returns[1:], order=(0,3))
results_ret_ma_3 = model_ret_ma_3.fit()
print(results_ret_ma_3.summary())
print('\nLLR test p-value = ' + str(LLR_test(results_ret_ma_2, results_ret_ma_3)))

In [ ]:
model_ret_ma_4 = ARMA(df.returns[1:], order=(0,4))
results_ret_ma_4 = model_ret_ma_4.fit()
print(results_ret_ma_4.summary())
print('\nLLR test p-value = ' + str(LLR_test(results_ret_ma_3, results_ret_ma_4)))

In [ ]:
model_ret_ma_5 = ARMA(df.returns[1:], order=(0,5))
results_ret_ma_5 = model_ret_ma_5.fit()
print(results_ret_ma_5.summary())
print('\nLLR test p-value = ' + str(LLR_test(results_ret_ma_4, results_ret_ma_5)))

In [ ]:
model_ret_ma_6 = ARMA(df.returns[1:], order=(0,6))
results_ret_ma_6 = model_ret_ma_6.fit()
print(results_ret_ma_6.summary())
print('\nLLR test p-value = ' + str(LLR_test(results_ret_ma_5, results_ret_ma_6)))

In [ ]:
model_ret_ma_7 = ARMA(df.returns[1:], order=(0,7))
results_ret_ma_7 = model_ret_ma_7.fit()
print(results_ret_ma_7.summary())
print('\nLLR test p-value = ' + str(LLR_test(results_ret_ma_6, results_ret_ma_7)))

In [ ]:
LLR_test(results_ret_ma_3, results_ret_ma_7, DF=2)

### Residuals of Returns

In [ ]:
df['res_ret_ma_7'] = results_ret_ma_7.resid[1:]

In [ ]:
print('The mean of the residual is ' + str(round(df.res_ret_ma_7.mean(),3)) + '\nThe variance of the residual is ' + str(round(df.res_ret_ma_7.var(),3)))

In [ ]:
from math import sqrt
round(sqrt(df.res_ret_ma_7.var()),3)

In [ ]:
df.res_ret_ma_7[1:].plot(figsize=(20,5))
plt.title('Residuals of Returns', size= 24)
plt.show()

In [ ]:
sts.adfuller(df.res_ret_ma_7[2:])

In [ ]:
sgt.plot_acf(df.res_ret_ma_7[2:],zero=False, lags=40)
plt.title('ACF of Residuals for Returns', size = 24)
plt.show()

### Normalized Returns

In [ ]:
bench_ret = df.returns.iloc[1]
df['norm_ret'] = df.returns.div(bench_ret).mul(100)

In [ ]:
sgt.plot_acf(df.norm_ret[1:],zero=False, lags=40)
plt.title('ACF of Normalized Returns', size = 24)
plt.show()

In [ ]:
model_norm_ret_ma_7 = ARMA(df.norm_ret[1:], order = (0,7))
results_norm_ret_ma_7 = model_norm_ret_ma_7.fit()
results_norm_ret_ma_7.summary()

In [ ]:
df['res_norm_ret_ma_7'] = results_ret_ma_7.resid[1:]

In [ ]:
df.res_norm_ret_ma_7[1:].plot(figsize=(20,5))
plt.title('Residuals of Normalized Returns', size= 24)
plt.show()

In [ ]:
sgt.plot_acf(df.res_norm_ret_ma_7[2:],zero=False, lags=40)
plt.title('ACF of Residuals for Normalized Returns', size = 24)
plt.show()

### MA Models for Pollutants 

In [ ]:
sgt.plot_acf(df.PM2,zero=False, lags=40)
plt.title('ACF for Pollutants', size = 24)
plt.show()

In [ ]:
model_ma_1 = ARMA(df.PM2, order = (0,1))
results_ma_1 = model_ma_1.fit()
results_ma_1.summary()

### ARMA(1,1)

In [ ]:
model_ret_ar_1_ma_1 = ARMA(df.returns[1:], order = (1,1))
results_ret_ar_1_ma_1 = model_ret_ar_1_ma_1.fit()
results_ret_ar_1_ma_1.summary()

In [ ]:
model_ret_ar_1 = ARMA(df.returns[1:], order = (1,0))
model_ret_ma_1 = ARMA(df.returns[1:], order = (0,1))

In [ ]:
print('\nARMA vs AR', LLR_test(results_ret_ar_1, results_ret_ar_1_ma_1))
print('\nARMA vs MA', LLR_test(results_ret_ma_1, results_ret_ar_1_ma_1))

### Higher-Lag ARMA Models

In [ ]:
model_ret_ar_3_ma_3 = ARMA(df.returns[1:], order = (3,3))
results_ret_ar_3_ma_3 = model_ret_ar_3_ma_3.fit()

In [ ]:
LLR_test(results_ret_ar_1_ma_1, results_ret_ar_3_ma_3,  DF = 4)

In [ ]:
results_ret_ar_3_ma_3.summary()

In [ ]:
model_ret_ar_3_ma_2 = ARMA(df.returns[1:], order = (3,2))
results_ret_ar_3_ma_2 = model_ret_ar_3_ma_2.fit()
results_ret_ar_3_ma_2.summary()

In [ ]:
model_ret_ar_2_ma_3 = ARMA(df.returns[1:], order = (2,3))
results_ret_ar_2_ma_3 = model_ret_ar_2_ma_3.fit()
results_ret_ar_2_ma_3.summary()

In [ ]:
LLR_test(results_ret_ar_2_ma_3, results_ret_ar_3_ma_3)

In [ ]:
model_ret_ar_3_ma_1 = ARMA(df.returns[1:], order = (3,1))
results_ret_ar_3_ma_1 = model_ret_ar_3_ma_1.fit()
results_ret_ar_3_ma_1.summary()

In [ ]:
LLR_test(results_ret_ar_3_ma_1, results_ret_ar_3_ma_2)

In [ ]:
model_ret_ar_2_ma_2 = ARMA(df.returns[1:], order = (2,2))
results_ret_ar_2_ma_2 = model_ret_ar_2_ma_2.fit()
results_ret_ar_2_ma_2.summary()

In [ ]:
model_ret_ar_1_ma_3 = ARMA(df.returns[1:], order = (1,3))
results_ret_ar_1_ma_3 = model_ret_ar_1_ma_3.fit()
results_ret_ar_1_ma_3.summary()

In [ ]:
print('\n ARMA (3,2): \tLL = ', results_ret_ar_3_ma_2.llf, results_ret_ar_3_ma_2.aic)
print('\n ARMA (1,3): \tLL = ', results_ret_ar_1_ma_3.llf, results_ret_ar_1_ma_3.aic)

# Residuals for Returns

---



In [ ]:
df['res_ret_ar_3_ma_2'] = results_ret_ar_3_ma_2.resid[1:]
df.res_ret_ar_3_ma_2.plot(figsize=(20,5))
plt.title('Residual for Returns', size=24)
plt.show()

In [ ]:
sgt.plot_acf(df.res_ret_ar_3_ma_2[2:], zero=False, lags = 40)
plt.title('ACF of Residuals for Returns', size=24)
plt.show()


Re-evaluating Model Selection 

In [ ]:
model_ret_ar_5_ma_5 = ARMA(df.returns[1:], order = (5,5))
results_ret_ar_5_ma_5 = model_ret_ar_5_ma_5.fit()
results_ret_ar_5_ma_5.summary()

In [ ]:
model_ret_ar_5_ma_1 = ARMA(df.returns[1:], order = (5,1))
results_ret_ar_5_ma_1 = model_ret_ar_5_ma_1.fit()
results_ret_ar_5_ma_1.summary()

In [ ]:
model_ret_ar_1_ma_5 = ARMA(df.returns[1:], order = (1,5))
results_ret_ar_1_ma_5 = model_ret_ar_1_ma_5.fit()
results_ret_ar_1_ma_5.summary()

In [ ]:
print('\n ARMA (5,1): \tLL = ', results_ret_ar_5_ma_1.llf, '\t AIC = ', results_ret_ar_5_ma_1.aic)
print('\n ARMA (1,5): \tLL = ', results_ret_ar_1_ma_5.llf,  '\t AIC = ', results_ret_ar_1_ma_5.aic)

In [ ]:
print('\n ARMA (3,2): \tLL = ', results_ret_ar_3_ma_2.llf, '\t AIC = ', results_ret_ar_3_ma_2.aic)

Residuals for the New Model

In [ ]:
df['res_ret_ar_5_ma_1'] = results_ret_ar_5_ma_1.resid

In [ ]:
sgt.plot_acf(df.res_ret_ar_5_ma_1[1:], zero=False, lags = 40)
plt.title('ACF of Residuals for Returns', size=24)
plt.show()

ARMA Model for Pollutant

In [ ]:
sgt.plot_acf(df.PM2, unbiased=True, zero=False, lags = 40)
plt.title('Autocorrelation Function for Pollutant', size=20)
plt.show()

In [ ]:
sgt.plot_pacf(df.PM2, alpha=0.05, zero=False, lags = 40, method=('ols'))
plt.title('Partial Autocorrelation Function for Pollutant', size=20)
plt.show()

In [ ]:
model_ar_1_ma_1 = ARMA(df.PM2, order = (1,1))
results_ar_1_ma_1 = model_ar_1_ma_1.fit()
results_ar_1_ma_1.summary()

In [ ]:
df['res_ar_1_ma_1'] = results_ar_1_ma_1.resid

In [ ]:
sgt.plot_acf(df.res_ar_1_ma_1, zero=False, lags = 40)
plt.title('ACF of Residuals for Pollutant', size=20)
plt.show()

In [ ]:
model_ar_6_ma_6 = ARMA(df.PM2, order = (6,6))
results_ar_6_ma_6 = model_ar_6_ma_6.fit()
results_ar_6_ma_6.summary()

In [ ]:
model_ar_5_ma_6 = ARMA(df.PM2, order = (5,6))
results_ar_5_ma_6 = model_ar_5_ma_6.fit()
results_ar_5_ma_6.summary()

In [ ]:
model_ar_6_ma_1 = ARMA(df.PM2, order = (6,1))
results_ar_6_ma_1 = model_ar_6_ma_1.fit()
results_ar_6_ma_1.summary()

In [ ]:
print('\n ARMA (5,6): \tLL = ', results_ar_5_ma_6.llf, '\t AIC = ', results_ar_5_ma_6.aic)
print('\n ARMA (6,1): \tLL = ', results_ar_6_ma_1.llf,  '\t AIC = ', results_ar_6_ma_1.aic)

In [ ]:
df['res_ar_5_ma_6'] = results_ar_5_ma_6.resid
sgt.plot_acf(df.res_ar_5_ma_6, zero=False, lags = 40)
plt.title('ACF of Residuals for Pollutant', size=20)
plt.show()

In [ ]:
print('\n ARMA (5,6): \tLL = ', results_ar_5_ma_6.llf, '\t AIC = ', results_ar_5_ma_6.aic)
print('\n ARMA (5,1): \tLL = ', results_ret_ar_5_ma_1.llf,  '\t AIC = ', results_ret_ar_5_ma_1.aic)

ARIMA(1,1)

In [ ]:
model_ar_1_i_1_ma_1 = ARIMA(df.PM2, order=(1,1,1))
results_ar_1_i_1_ma_1 = model_ar_1_i_1_ma_1.fit()
results_ar_1_i_1_ma_1.summary()

Residuals of the ARIMA(1,1,1)

In [ ]:
df['res_ar_1_i_1_ma_1'] = results_ar_1_i_1_ma_1.resid
sgt.plot_acf(df.res_ar_1_i_1_ma_1[1:], zero=False, lags = 40)
plt.title('ACF of Residuals for ARIMA(1,1,1)', size=20)
plt.show()

Higher-Lag ARIMA Models

In [ ]:
model_ar_1_i_1_ma_2 = ARIMA(df.PM2, order=(1,1,2))
results_ar_1_i_1_ma_2 = model_ar_1_i_1_ma_2.fit()

model_ar_1_i_1_ma_3 = ARIMA(df.PM2, order=(1,1,3))
results_ar_1_i_1_ma_3 = model_ar_1_i_1_ma_3.fit()

model_ar_2_i_1_ma_1 = ARIMA(df.PM2, order=(2,1,1))
results_ar_2_i_1_ma_1 = model_ar_2_i_1_ma_1.fit()

model_ar_3_i_1_ma_1 = ARIMA(df.PM2, order=(3,1,1))
results_ar_3_i_1_ma_1 = model_ar_3_i_1_ma_1.fit()

model_ar_3_i_1_ma_2 = ARIMA(df.PM2, order=(3,1,2))
results_ar_3_i_1_ma_2 = model_ar_3_i_1_ma_2.fit()

In [ ]:
print('\n ARMA (1,1,1): \tLL = ', results_ar_1_i_1_ma_1.llf, '\t AIC = ', results_ar_1_i_1_ma_1.aic)
print('\n ARMA (1,1,2): \tLL = ', results_ar_1_i_1_ma_2.llf, '\t AIC = ', results_ar_1_i_1_ma_2.aic)
print('\n ARMA (1,1,3): \tLL = ', results_ar_1_i_1_ma_3.llf, '\t AIC = ', results_ar_1_i_1_ma_3.aic)
print('\n ARMA (2,1,1): \tLL = ', results_ar_2_i_1_ma_1.llf, '\t AIC = ', results_ar_2_i_1_ma_1.aic)
print('\n ARMA (3,1,1): \tLL = ', results_ar_3_i_1_ma_1.llf, '\t AIC = ', results_ar_3_i_1_ma_1.aic)
print('\n ARMA (3,1,2): \tLL = ', results_ar_3_i_1_ma_2.llf, '\t AIC = ', results_ar_3_i_1_ma_2.aic)

In [ ]:
print('\nLLR test p-value = ' + str(LLR_test(results_ar_1_i_1_ma_2, results_ar_1_i_1_ma_3)))

In [ ]:
print('\nLLR test p-value = ' + str(LLR_test(results_ar_1_i_1_ma_1, results_ar_1_i_1_ma_3, DF=2)))

In [ ]:
df['res_ar_1_i_1_ma_3'] = results_ar_1_i_1_ma_3.resid.iloc[:]
sgt.plot_acf(df.res_ar_1_i_1_ma_3[1:], zero=False, lags = 40)
plt.title('ACF of Residuals for ARIMA(1,1,3)', size=20)
plt.show()

In [ ]:
model_ar_5_i_1_ma_1 = ARIMA(df.PM2, order=(5,1,1))
results_ar_5_i_1_ma_1 = model_ar_5_i_1_ma_1.fit()

model_ar_6_i_1_ma_3 = ARIMA(df.PM2, order=(6,1,3))
results_ar_6_i_1_ma_3 = model_ar_6_i_1_ma_3.fit()

In [ ]:
print('\n ARMA (1,1,3): \tLL = ', results_ar_1_i_1_ma_3.llf, '\t AIC = ', results_ar_1_i_1_ma_3.aic)
print('\n ARMA (5,1,1): \tLL = ', results_ar_5_i_1_ma_1.llf, '\t AIC = ', results_ar_5_i_1_ma_1.aic)
print('\n ARMA (6,1,3): \tLL = ', results_ar_6_i_1_ma_3.llf, '\t AIC = ', results_ar_6_i_1_ma_3.aic)

In [ ]:
print('\nLLR test p-value = ' + str(LLR_test(results_ar_1_i_1_ma_3, results_ar_6_i_1_ma_3, DF=5)))

In [ ]:
print('\nLLR test p-value = ' + str(LLR_test(results_ar_5_i_1_ma_1, results_ar_6_i_1_ma_3, DF=3)))

In [ ]:
df['res_ar_5_i_1_ma_1'] = results_ar_5_i_1_ma_1.resid.iloc[:]
sgt.plot_acf(df.res_ar_5_i_1_ma_1[1:], zero=False, lags = 40)
plt.title('ACF of Residuals for ARIMA(5,1,1)', size=20)
plt.show()

Model with Higher Levels of Integration 

In [ ]:
df['delta_changes'] = df.PM2.diff(1)

In [ ]:
model_delta_ar_1_i_1_ma_1 = ARIMA(df.delta_changes[1:], order=(1,0,1))
results_delta_ar_1_i_1_ma_1 = model_delta_ar_1_i_1_ma_1.fit()
results_delta_ar_1_i_1_ma_1.summary()

In [ ]:
sts.adfuller(df.delta_changes[1:])

In [ ]:
model_ar_1_i_2_ma_1 = ARIMA(df.delta_changes[1:], order=(1,0,2))
results_ar_1_i_2_ma_1 = model_ar_1_i_2_ma_1.fit()
results_ar_1_i_2_ma_1.summary()

In [ ]:
df['res_ar_1_i_2_ma_1'] = results_ar_1_i_2_ma_1.resid.iloc[:]
sgt.plot_acf(df.res_ar_1_i_2_ma_1[2:], zero=False, lags = 40)
plt.title('ACF of Residuals for ARIMA(1,2,1)', size=20)
plt.show()

ARIMAX

In [ ]:
model_ar_1_i_1_ma_1.Xpm = ARIMA(df.PM2, exog = df.pm, order=(1,1,1))
results_ar_1_i_1_ma_1.Xpm = model_ar_1_i_1_ma_1.Xpm.fit()
results_ar_1_i_1_ma_1.Xpm.summary()

SARIMAX

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
model_sarimax = SARIMAX(df.PM2, exog=df.pm, order =(1,0,1), seasonal_order=(2,0,1,5))
results_sarimax = model_sarimax.fit()
results_sarimax.summary()

Creating Returns for Volatility of ARCH Model

Creating Returns

In [ ]:
df['returns'] = df.PM2.pct_change(1)*100

Creating Squared Returns

In [ ]:
df['sq_returns'] = df.returns.mul(df.returns)

Returns vs Squared Returns

In [ ]:
df.returns.plot(figsize=(20,5))
plt.title('Returns', size=24)
plt.show()

In [ ]:
df.sq_returns.plot(figsize=(20,5))
plt.title('Volatility', size=24)
plt.show()

In [ ]:
sgt.plot_pacf(df.returns[1:], alpha=0.05, lags=40, zero=False, method=('ols'))
plt.title('PACF of Returns', size=20)
plt.show()

In [ ]:
sgt.plot_pacf(df.sq_returns[1:], alpha=0.05, lags=40, zero=False, method=('ols'))
plt.title('PACF of Squared Returns', size=20)
plt.show()

The arch_model() Method

In [ ]:

!pip install arch
from arch import arch_model

In [ ]:
model_arch_1 = arch_model(df.returns[1:])
results_arch_1 = model_arch_1.fit(update_freq=5)
results_arch_1.summary()

The Simple ARCH(1)

In [ ]:
model_arch_1 = arch_model(df.returns[1:], mean='Constant', vol='ARCH', p=1)
results_arch_1 = model_arch_1.fit(update_freq=5)
results_arch_1.summary()

In [ ]:
model_arch_1 = arch_model(df.returns[1:], mean='AR', lags=[2,3,6], vol='ARCH', p=1, dist='ged')
results_arch_1 = model_arch_1.fit(update_freq=5)
results_arch_1.summary()

Higher-Lag ARCH Models

In [ ]:
model_arch_2 = arch_model(df.returns[1:], mean='Constant', vol='ARCH', p=1)
results_arch_2 = model_arch_2.fit(update_freq=5)
results_arch_2.summary()

In [ ]:
model_arch_3 = arch_model(df.returns[1:], mean='Constant', vol='ARCH', p=1)
results_arch_3 = model_arch_3.fit(update_freq=5)
results_arch_3.summary()

The Simple GARCH Model

In [ ]:
model_garch_1_1 = arch_model(df.returns[1:], mean='Constant', vol='GARCH', p=1, q=1)
results_garch_1_1 = model_garch_1_1.fit(update_freq=5)
results_garch_1_1.summary()

Higher-Lag GARCH Models

In [ ]:
model_garch_1_2 = arch_model(df.returns[1:], mean='Constant', vol='GARCH', p=1, q=2)
results_garch_1_2 = model_garch_1_2.fit(update_freq=5)
results_garch_1_2.summary()

In [ ]:
model_garch_1_3 = arch_model(df.returns[1:], mean='Constant', vol='GARCH', p=1, q=3)
results_garch_1_3 = model_garch_1_3.fit(update_freq=5)
results_garch_1_3.summary()

In [ ]:
model_garch_2_1 = arch_model(df.returns[1:], mean='Constant', vol='GARCH', p=2, q=1)
results_garch_2_1 = model_garch_2_1.fit(update_freq=5)
results_garch_2_1.summary()

In [ ]:
model_garch_3_1 = arch_model(df.returns[1:], mean='Constant', vol='GARCH', p=3, q=1)
results_garch_3_1 = model_garch_3_1.fit(update_freq=5)
results_garch_3_1.summary()